In [11]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../../data/'
group_dir_ = os.path.join('../../../HPCF/all/test_0.8/data/groups/high')
folder = 'all (user)'
file_name = 'normalized_to_rating_filter_track_5_user_100.pkl'

In [12]:
predictions = pd.read_pickle(os.path.join(dir_, folder, 'topN_pred_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, folder, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,rating,count
26,0,9,4.75,15
37,0,15,2.50,3
45,0,22,3.50,6
64,0,28,2.75,4
75,0,33,0.25,1


In [13]:
all_user = pd.read_pickle(os.path.join('../../../HPCF/all/test_0.8/data/', file_name[:-3] + 'pkl'))
df = pd.read_pickle(os.path.join('../../../HPCF/all/test_0.8/data/', file_name[:-3] + 'pkl'))

In [8]:
predictions[:5]

,uid,tid,rating
0,0,9,0.847839
1,0,15,0.594322
2,0,22,0.446187
3,0,28,0.721104
4,0,33,0.597911


In [9]:
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,rating,count
26,0,9,4.75,15
37,0,15,2.50,3
45,0,22,3.50,6
64,0,28,2.75,4
75,0,33,0.25,1


In [10]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

OSError: ../../../../HPCF/all/test_0.8/data/groups/high/2.csv not found.

In [7]:
num_user = len(test['uid'].unique())
num_user

953

In [8]:
pop_dict = {}
for i in all_user['tid'].unique():
    pop_dict[i] = len(all_user[all_user['tid']==i])/num_user

In [9]:
top_n_size = 50
top_n_lists_g = []
top_n_items_g = []
for groups in groups_n:

    group_size = len(groups[0])
    top_n_lists = []
    top_n_items = pd.DataFrame()

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 
        
        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)

        if len(top_n_items) == 0:
            top_n_items = rating_table
        else:
            top_n_items = top_n_items.set_index('tid').add(rating_table.set_index('tid'), fill_value=0).reset_index()

    top_n_lists_g.append(top_n_lists)   
    top_n_items_g.append(top_n_items)

COV = []    
NOV = []
HitNOV = []
HitCOV = []
for i in range(len(groups_n)):
    groups = groups_n[i]
    top_n_lists = top_n_lists_g[i]
    top_n_items = top_n_items_g[i]

    nov = 0
    hitNov = 0
    hits = np.zeros(len(df['tid'].unique()), dtype=bool)
    for j in (range(len(groups))):
        group = groups[j]
        top_n_list = top_n_lists[j]

        for k in range(len(group)):
            for l in range(top_n_size):
                uid = group[k]
                tid = top_n_list[l]
                t = test[test['uid'] == uid]
                t = t[t['tid'] == tid]
                nov += (1-pop_dict[tid])
                if len(t) > 0:
                    hits[int(tid)] = True
                    hitNov += (1-pop_dict[tid])
    NOV.append(nov/top_n_size/(len(groups)*len(groups[0])))
    COV.append(len(top_n_items)/len(df['tid'].unique()))
    HitNOV.append(hitNov/top_n_size/(len(groups)*len(groups[0])))
    HitCOV.append(list(hits).count(True)/len(df['tid'].unique()))
print(NOV)
print(HitNOV)
print(COV)
print(HitCOV)

[0.6198342747975399, 0.6467641602578508, 0.6632071353620055, 0.6784287435389165, 0.6867783379827539, 0.6957861935196638, 0.7034318636282783, 0.7084080680890846, 0.713771894422468]
[0.09221926877750569, 0.08469554283386675, 0.08213525708289614, 0.07961645485989699, 0.07146503553178263, 0.07094175149789034, 0.0693057672889981, 0.06764955371601047, 0.0622518901175773]
[0.0018722194368110073, 0.002525909613053495, 0.0035223111438308784, 0.0037317458604910927, 0.004150615293811522, 0.004366396517037197, 0.004258505905424359, 0.004283891931686204, 0.0041188827609842165]
[0.0017072102661090203, 0.0023609004423515074, 0.0031224812302068326, 0.003338262453432508, 0.0035921227160509495, 0.0035730831963545666, 0.0035413506635272613, 0.0035286576503963394, 0.003306529920605203]
